# Executive Order Final Dataset

Merge some results into the final dataset.

In [26]:
import pandas

## Combine Datasets

Combine some datasets so we can understand which presidents and parties are in each cluster.

In [ ]:
clusters_df = pandas.read_csv('data/executive_orders/clusters.csv')
clusters_df['name'] = clusters_df['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
clusters_df.drop(columns=['Unnamed: 0'], inplace=True)

# add sentiment which also contains the president
sentiment_df = pandas.read_csv('data/executive_orders/sentiment.csv')
combined_df = clusters_df.merge(sentiment_df, on='name')

# join with president's metadata
presidents_df = pandas.read_csv('data/executive_orders/presidents.csv')
combined_df = combined_df.merge(presidents_df, on='president')

# add cluster names
cluster_analysis_df = pandas.read_json('data/executive_orders/refined_clusters.json')
cluster_analysis_df = cluster_analysis_df[['cluster', 'title']].rename(columns={'title': 'cluster_title'})
combined_df = combined_df.merge(cluster_analysis_df, on='cluster')

# drop some stuff we don't care about
combined_df.drop(columns=['path', 'duration'], inplace=True)

# oops
combined_df = combined_df.rename(columns={
    'initial_idelogical_tilt': 'initial_ideological_tilt',
    'revised_idelogical_tilt': 'revised_ideological_tilt'
})

# write to csv
combined_df.to_csv('data/executive_orders/combined.csv', index=False)

combined_df

,cluster,tsne_1,tsne_2,name,president,president_byline,date,title,initial_joy,initial_sadness,...,revised_ideological_tilt,revised_immigration,revised_executive_power,revised_federalism,revised_security_liberty,revised_economic_regulation,revised_social_inclusion,presidential order,party,cluster_title
0,40,-24.312270,12.092871,executive-order-5855-assignment-frequencies-go...,Herbert Hoover,31st President of the United States: 1929 ‐ 1933,1932-06-06,Executive Order 5855-Assignment of Frequencies...,1.0,1.0,...,0.0,0.0,3.0,3.0,1.0,2.0,0.0,31,Republican,Executive Branch Organization: Authority and C...
1,43,-14.963946,45.930885,executive-order-6756-c-approval-code-fair-comp...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1934-06-29,Executive Order 6756-C-Approval of Code of Fai...,1.0,1.0,...,3.0,0.0,3.0,4.0,0.0,5.0,0.0,32,Democratic,Executive Branch Advisory Groups: Formation an...
2,0,-43.212210,-42.001114,executive-order-taking-military-possession-rai...,Abraham Lincoln,16th President of the United States: 1861 ‐ 1865,1862-05-25,Executive Order-Taking Military Possession of ...,1.0,3.0,...,0.0,0.0,3.0,4.0,3.0,4.0,0.0,16,Republican,Civil War to Early 20th Century Presidential a...
3,0,-47.932255,-36.022755,executive-order-concerning-blockade,Abraham Lincoln,16th President of the United States: 1861 ‐ 1865,1862-11-12,Executive Order-Concerning Blockade,1.0,1.0,...,0.0,0.0,1.0,0.0,2.0,0.0,0.0,16,Republican,Civil War to Early 20th Century Presidential a...
4,11,-25.279959,-35.540390,executive-order-6801-designating-the-honorable...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1934-07-27,Executive Order 6801-Designating the Honorable...,1.0,1.0,...,0.0,0.0,2.0,3.0,0.0,0.0,0.0,32,Democratic,Adjustments to Existing Executive Orders and R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10434,19,-3.141809,-89.441460,executive-order-7935-exemption-j-humbird-smith...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1938-07-16,Executive Order 7935-Exemption of J. Humbird S...,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,32,Democratic,Presidential Exemptions to Mandatory Retirement
10435,24,-77.745210,42.446953,executive-order-7280-revocation-part-executive...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1936-02-01,Executive Order 7280-A-Revocation in Part of E...,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32,Democratic,Land Withdrawal and Resurvey Orders in Western...
10436,29,7.557537,-15.218990,executive-order-10623-amendment-certain-provis...,Dwight D. Eisenhower,34th President of the United States: 1953 ‐ 1961,1955-07-23,Executive Order 10623-Amendment of Certain Pro...,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34,Republican,Military and Veteran Personnel: Regulation and...
10437,14,-63.507320,0.855812,executive-order-5472-withdrawal-public-lands-p...,Herbert Hoover,31st President of the United States: 1929 ‐ 1933,1930-10-27,Executive Order 5472-Withdrawal of Public Land...,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,31,Republican,California and Nevada Public Land Management


In [41]:
combined = pandas.read_csv('data/executive_orders/combined.csv')

# group by cluster and create a column of array of unique presidents and parties
grouped = combined.groupby('cluster').agg({
    'president': lambda x: list(set(x)),
    'party': lambda x: list(set(x)),
    'cluster_title': 'first'
}).reset_index()
grouped

,cluster,president,party,cluster_title
0,0,"[Gerald R. Ford, Chester A. Arthur, George Was...","[Whig, Democratic, Republican, No Party]",Civil War to Early 20th Century Presidential a...
1,1,"[Richard Nixon, William J. Clinton, George W. ...","[Democratic, Republican]",Executive Order Amendments: Government Structu...
2,2,"[Andrew Johnson, Warren G. Harding, Dwight D. ...","[Democratic, Republican]",Alaskan Land Reallocations for Development and...
3,3,"[Richard Nixon, Dwight D. Eisenhower, Gerald R...","[Democratic, Republican]",Government Authorized Tax Return Inspections
4,4,"[Richard Nixon, William J. Clinton, Dwight D. ...","[Democratic, Republican]",Government Land Management and Railway Labor R...
5,5,"[Richard Nixon, William J. Clinton, George W. ...","[Democratic, Republican]",Delegation of Presidential Authority to Govern...
6,6,"[Joseph R. Biden, Jr., Gerald R. Ford, George ...","[Democratic, Republican]",Executive Actions: Directing National Policy a...
7,7,"[Richard Nixon, William J. Clinton, George W. ...","[Democratic, Republican]",Restricted Air and Sea Zones for National Defense
8,8,"[William J. Clinton, George W. Bush, Joseph R....","[Democratic, Republican]",Presidential Directives on National and Intern...
9,9,"[Franklin D. Roosevelt, Herbert Hoover]","[Democratic, Republican]",Presidential Exemptions to Mandatory Retirement
